In [6]:
import folium
import os
import json
import pandas as pd
#from shapely.geometry import MultiPolygon, shape, mapping

#change dir to the location of the data_bourgogne.json file which contains only locations of Burgundy wine
os.chdir('/Users/Gabriel/Documents/GitHub/Projet_informatique_ENSAE/jsons')
%run '/Users/Gabriel/Documents/GitHub/Projet_informatique_ENSAE/cartes/scripts/function_addlayer.py'

In [7]:
# Open the file

with open('data_bourgogne_wtype.json') as json_file:
    data = json.load(json_file)
    
file = open("/Users/Gabriel/Documents/GitHub/Projet_informatique_ENSAE/cartes/scripts/grandscrus.txt",'r')
grandscrus = file.read().split('\n')

for key in data.keys():
    for gc in grandscrus:
        if data[key]['properties']['appellation'] == gc :
            data[key]['properties']['Grand Cru'] = 1
            
df = pd.read_csv('/Users/Gabriel/Documents/GitHub/Projet_informatique_ENSAE/cartes/scripts/cote_nuit.csv', header = None)
df.columns = ['appellation', 'type vin']

appellation = [x.lower() for x in df['appellation']]
a = ['le chapitre', 'montrecul','hautes côtes de nuits']

In [9]:
data_cote2nuit = {}
data_rouge = {}
data_blanc = {}

for key in data.keys():
    if (data[key]['properties']['appellation'].lower() in appellation) or (data[key]['properties']['climat'].lower() in a) :
        data_cote2nuit[key] = data[key]
        
for key in data_cote2nuit.keys():
    if data_cote2nuit[key]['properties']['type_vin'] == 'rouge':
        if 'mousseux' not in data_cote2nuit[key]['properties']['appellation'].lower() and "Crémant" not in data_cote2nuit[key]['properties']['appellation'] :
            data_rouge[key] = data_cote2nuit[key]
        
    if data_cote2nuit[key]['properties']['type_vin'] == 'blanc':
        if 'mousseux' not in data_cote2nuit[key]['properties']['appellation'].lower() and "Crémant" not in data_cote2nuit[key]['properties']['appellation']:
            data_blanc[key] = data_cote2nuit[key]

In [10]:
style_redwine = {'fillColor': '#DC1A40', 'color': '#464140', 'fill_opacity': 1, 'line_opacity' : 0.5}
style_whitewine = {'fillColor': '#EAE010', 'color': '#464140', 'fill_opacity': 1, 'line_opacity' : 0.5}

os.chdir('/Users/Gabriel/Documents/GitHub/Projet_informatique_ENSAE')

wine_map = folium.Map(location=[47.2334, 4.96],
                      height=500,
                      tiles = "Stamen Terrain",
                      zoom_start=12,
                      prefer_canvas=True)

add_layertomap_no_image(data_rouge, style_redwine, 'Côte de Nuit Rouge', wine_map)
add_layertomap_no_image(data_blanc, style_whitewine, 'Côte de Nuit Blanc', wine_map, show = False)
folium.LayerControl().add_to(wine_map)

#wine_map.save(os.path.join('results', 'cote2nuit.html'))
wine_map.save('/Users/Gabriel/Documents/GitHub/Projet_informatique_ENSAE/cartes/cote2nuit.html')
#display(wine_map)